# Matrix Factorization

In [2]:
import pandas as pd 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse 

class MF(object):
    """docstring for CF"""
    def __init__(self, Y_data, K, lam = 0.1, Xinit = None, Winit = None, 
                 learning_rate = 0.5, max_iter = 1000, print_every = 100, user_based = 0):
        self.Y_raw = Y_data.copy()
        self.Y_data = Y_data.copy()
        self.K = K
        self.lam = lam
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.print_every = print_every
        self.user_based = user_based
        # number of users and items. Remember to add 1 since id starts from 0
        self.n_users = int(np.max(Y_data[:, 0])) + 1 
        self.n_items = int(np.max(Y_data[:, 1])) + 1
        
        if Xinit is None: 
            self.X = np.random.randn(self.n_items, K)
        else:
            self.X = Xinit 
        
        if Winit is None: 
            self.W = np.random.randn(K, self.n_users)
        else: 
            self.W = Winit
        
        # item biases
        self.b = np.random.randn(self.n_items)
        self.d = np.random.randn(self.n_users)
        #self.all_users = self.Y_data[:,0] # all users (may be duplicated)
        self.n_ratings = Y_data.shape[0]
#         self.mu = np.mean(Y_data[:, 2])
        self.mu = 0
 

    def normalize_Y(self):
        if self.user_based:
            user_col = 0
            item_col = 1
            n_objects = self.n_users
        else:
            user_col = 1
            item_col = 0 
            n_objects = self.n_items
        self.muu = np.zeros(n_objects)
        self.Y_data = self.Y_raw
        return
    
        if self.user_based:
            user_col = 0
            item_col = 1
            n_objects = self.n_users
        else:
            user_col = 1
            item_col = 0 
            n_objects = self.n_items

        users = self.Y_data[:, user_col] 
        self.muu = np.zeros((n_objects,))
        for n in range(n_objects):
            # row indices of rating done by user n
            # since indices need to be integers, we need to convert
            ids = np.where(users == n)[0].astype(np.int32)
            # indices of all ratings associated with user n
            item_ids = self.Y_data[ids, item_col] 
            # and the corresponding ratings 
            ratings = self.Y_data[ids, 2]
            # take mean
            m = np.mean(ratings) 

            if np.isnan(m):
                m = 0 # to avoid empty array and nan value
            self.muu[n] = m
            # normalize
            self.Y_data[ids, 2] = ratings - m
            
            
    def loss(self):
        L = 0 
        for i in range(self.n_ratings):
            # user, item, rating
            n, m, rate = int(self.Y_data[i, 0]), int(self.Y_data[i, 1]), self.Y_data[i, 2]
            L += 0.5*(self.X[m, :].dot(self.W[:, n]) + self.b[m] + self.d[n] + self.mu - rate)**2
            
        # regularization, don't ever forget this 
        L /= self.n_ratings
        L += 0.5*self.lam*(np.sum(self.X**2) + np.sum(self.W**2))
#         L += 0.5*self.lam*(np.linalg.norm(self.X, 'fro')**2 + np.linalg.norm(self.W, 'fro')**2 + \
#                           np.linalg.norm(self.b) + np.linalg.norm(self.d))
        return L 

    
    def get_items_rated_by_user(self, user_id):
        """
        get all items which are rated by user n, and the corresponding ratings
        """
        # y = self.Y_data_n[:,0] # all users (may be duplicated)
        # item indices rated by user_id
        # we need to +1 to user_id since in the rate_matrix, id starts from 1 
        # while index in python starts from 0
        ids = np.where(self.Y_data[:,0] == user_id)[0] 
        item_ids = self.Y_data[ids, 1].astype(np.int32) # index starts from 0 
        ratings = self.Y_data[ids, 2]
        return (item_ids, ratings)
        
        
    def get_users_who_rate_item(self, item_id):
        """
        get all users who rated item m and get the corresponding ratings
        """
        ids = np.where(self.Y_data[:,1] == item_id)[0] 
        user_ids = self.Y_data[ids, 0].astype(np.int32)
        ratings = self.Y_data[ids, 2]
        return (user_ids, ratings)
        
    def updateX(self): # and b 

            for m in range(self.n_items):
                user_ids, ratings = self.get_users_who_rate_item(m)

                Wm = self.W[:, user_ids]
                dm = self.d[user_ids]
                for i in range(30):
                    xm = self.X[m, :]

                    error = xm.dot(Wm) + self.b[m] + dm + self.mu - ratings 

                    grad_xm = error.dot(Wm.T)/self.n_ratings + self.lam*xm
                    grad_bm = np.sum(error)/self.n_ratings# + self.lam*self.b[m]
                    self.X[m, :] -= self.learning_rate*grad_xm.reshape(-1)
                    self.b[m]    -= self.learning_rate*grad_bm
    
    def updateW(self): # and d 
            for n in range(self.n_users):
                item_ids, ratings = self.get_items_rated_by_user(n)
                Xn = self.X[item_ids, :]
                bn = self.b[item_ids]
                for i in range(30):
                    wn = self.W[:, n]
                    error = Xn.dot(wn) + bn + self.mu + self.d[n] - ratings
                    grad_wn = Xn.T.dot(error)/self.n_ratings + self.lam*wn
                    grad_dn = np.sum(error)/self.n_ratings# + self.lam*self.d[n]
                    self.W[:, n] -= self.learning_rate*grad_wn.reshape(-1)
                    self.d[n]    -= self.learning_rate*grad_dn
    
    def fit(self):
        self.normalize_Y()
        for it in range(self.max_iter):
            self.updateW()
            self.updateX()
            if (it + 1) % self.print_every == 0:
                rmse_train = self.evaluate_RMSE(self.Y_raw)
                print ('iter =', it + 1, ', loss =', self.loss(), ', RMSE train =', rmse_train)
    
    
    def pred(self, u, i):
        """ 
        predict the rating of user u for item i 
        if you need the un
        """
        u = int(u)
        i = int(i)
        if self.user_based == 1:
            bias = self.muu[u]
        else:
            bias = self.muu[i]
        
        pred = self.X[i, :].dot(self.W[:, u]) + self.b[i] + self.d[u] + bias
        return max(0, min(5, pred))
        
    
    def pred_for_user(self, user_id):
        ids = np.where(self.Y_data_n[:, 0] == user_id)[0]
        items_rated_by_u = self.Y_data_n[ids, 1].tolist()              
        
        y_pred = self.X.dot(self.W[:, user_id])
        predicted_ratings= []
        for i in range(self.n_items):
            if i not in items_rated_by_u:
                predicted_ratings.append((i, y_pred[i]))
        
        return predicted_ratings
    
    def evaluate_RMSE(self, rate_test):
        n_tests = rate_test.shape[0]
        SE = 0 # squared error
        for n in range(n_tests):
            pred = self.pred(rate_test[n, 0], rate_test[n, 1])
#             print pred, rate_test[n, 2]
            SE += (pred - rate_test[n, 2])**2 

        RMSE = np.sqrt(SE/n_tests)
        return RMSE
        
r_cols = ['user_id', 'item_id', 'rating']
ratings = pd.read_csv('ex.dat', sep = ' ', names = r_cols, encoding='latin-1')
Y_data = ratings[r_cols].values


rs = MF(Y_data, K = 3, max_iter = 1000, print_every = 100, lam = 0.1)

rs.fit()
rs.pred(6, 1)
print (rs.evaluate_RMSE(Y_data))

iter = 100 , loss = 0.6771573758208562 , RMSE train = 0.6459393234199525
iter = 200 , loss = 0.6771573758208117 , RMSE train = 0.6459393234200664
iter = 300 , loss = 0.6771573758208117 , RMSE train = 0.6459393234200664
iter = 400 , loss = 0.6771573758208117 , RMSE train = 0.6459393234200664
iter = 500 , loss = 0.6771573758208117 , RMSE train = 0.6459393234200664
iter = 600 , loss = 0.6771573758208117 , RMSE train = 0.6459393234200664
iter = 700 , loss = 0.6771573758208117 , RMSE train = 0.6459393234200664
iter = 800 , loss = 0.6771573758208117 , RMSE train = 0.6459393234200664
iter = 900 , loss = 0.6771573758208117 , RMSE train = 0.6459393234200664
iter = 1000 , loss = 0.6771573758208117 , RMSE train = 0.6459393234200664
0.6459393234200664


# Áp dụng lên MovieLens 100k

In [3]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('ml-100k/ua.test', sep='\t', names=r_cols, encoding='latin-1')

rate_train = ratings_base[r_cols].values
rate_test = ratings_test[r_cols].values

# indices start from 0
rate_train[:, :2] -= 1
rate_test[:, :2] -= 1

In [4]:
rs = MF(rate_train, K = 50, lam = .01, print_every = 1, learning_rate = 50, max_iter = 30, user_based = 0)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print ('\nUser-based MF, RMSE =', RMSE)

iter = 1 , loss = 0.818103902724614 , RMSE train = 1.225483635870316
iter = 2 , loss = 0.5464814343455631 , RMSE train = 1.0443471441048047
iter = 3 , loss = 0.4857952426419337 , RMSE train = 0.9852527009697586
iter = 4 , loss = 0.4585134576241089 , RMSE train = 0.9573192380457018
iter = 5 , loss = 0.44384368332279084 , RMSE train = 0.9419068885175774
iter = 6 , loss = 0.43512619215278175 , RMSE train = 0.9326276930733748
iter = 7 , loss = 0.4295981769839635 , RMSE train = 0.9266985277945321
iter = 8 , loss = 0.4259244099940175 , RMSE train = 0.9227381224068802
iter = 9 , loss = 0.4233918071110596 , RMSE train = 0.9199981879446365
iter = 10 , loss = 0.4215920258321279 , RMSE train = 0.9180460588392063
iter = 11 , loss = 0.42027888531838353 , RMSE train = 0.9166191604289436
iter = 12 , loss = 0.4192979575973349 , RMSE train = 0.915548978737078
iter = 13 , loss = 0.4185492426500177 , RMSE train = 0.9147308084701189
iter = 14 , loss = 0.4179662640673648 , RMSE train = 0.9140913941884442
i

In [5]:
rs.d

array([2.87425574, 2.90956755, 2.43575752, 3.82127687, 2.42677735,
       2.58096518, 3.24403773, 3.09404566, 3.22352365, 3.17547943,
       2.71956369, 3.32369678, 2.5457139 , 3.08876842, 2.3977117 ,
       3.43923084, 2.37490832, 2.87583698, 2.82483003, 2.41485799,
       2.47121093, 2.81919494, 2.69991841, 3.39214434, 2.9044119 ,
       2.34770837, 2.66461924, 2.96983299, 3.03691808, 3.16362412,
       2.91628881, 2.576128  , 3.35957238, 3.43817998, 2.72372968,
       3.84658115, 3.03286887, 3.37514902, 3.11107059, 2.06389422,
       2.6922867 , 2.99235442, 2.98698215, 2.77744451, 3.16655642,
       3.26169598, 2.91086625, 2.65446019, 2.08925183, 2.87339138,
       2.46971884, 3.45886805, 3.15912036, 3.12660553, 2.93579076,
       3.01185858, 2.9940798 , 2.85551577, 3.12004642, 3.06299186,
       2.06803789, 2.52662237, 2.39935912, 2.68955285, 2.96802987,
       2.77852747, 3.34947892, 2.4296392 , 2.96067046, 2.72533759,
       2.53512482, 2.85349175, 2.68191118, 2.9110876 , 2.88835